In [31]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate


In [25]:
adult_census = pd.read_csv(r"C:\Users\joshk\Documents\Progamming\mach-learn datasets\adult census\phpMawTba.csv")
schema = pd.read_csv(r'c:\Users\joshk\Documents\Progamming\mach-learn datasets\adult census\feature_list.csv')
adult_census.head()





,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [6]:
# using a schema of extracting data type from the dataset columns we can determine if the data needs preprocesssing. As catorgorical data is not a numerical value and will need to be encoded.
schema

,Feature,Type
0,age,Continuous
1,workclass,Categorical
2,fnlwgt,Continuous
3,education,Categorical
4,education-num,Continuous
5,marital-status,Categorical
6,occupation,Categorical
7,relationship,Categorical
8,race,Categorical
9,sex,Categorical


In [28]:
target_name = 'class'
target = adult_census[target_name]
data = adult_census.drop(columns= [target_name])

In [13]:
# using ordinal encoder we transform catorgorical data to a numerical value
education_col = adult_census[['education']]
encoder = OrdinalEncoder().set_output(transform = 'pandas')
education_encoded = encoder.fit_transform(education_col)
education_encoded


,education
0,1.0
1,11.0
2,7.0
3,15.0
4,15.0
...,...
48837,7.0
48838,11.0
48839,11.0
48840,11.0


In [17]:
encoder.categories_

[array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
        'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
        'Masters', 'Preschool', 'Prof-school', 'Some-college'],
       dtype=object)]

In [14]:
# update the dataframe with encoded data
adult_census['education'] = education_encoded

In [15]:
adult_census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.0,Private,226802.0,1.0,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,11.0,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,7.0,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,160323.0,15.0,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18.0,NaN,103497.0,15.0,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [29]:
# using method pipeline guarantee that the oreder is executed correctly. ie encodeding then model fit. This will pass all catorgorical data through OneHotEncoder.
# it will only allow 500 interartions for convergence.
model = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"), LogisticRegression(max_iter=500)
)

In [30]:
# using function cross_validate() to divied data into k folds(subsets of data ) 1 fold is chosen as target the k-1 folds are the training data. carry out k iterations of this process.

cv_results = cross_validate(model, data, target)
cv_results

{'fit_time': array([0.88384581, 0.61229777, 0.60802674, 0.57704043, 0.6401186 ]),
 'score_time': array([0.03179336, 0.03376055, 0.03097868, 0.03489661, 0.0357759 ]),
 'test_score': array([0.86999693, 0.86948511, 0.86773137, 0.87080262, 0.87346437])}

In [32]:
scores = cv_results["test_score"]
# scores.mean give the probiltiy of prediction to 3 decimal places,  with scores.std giving the standard deviation to 3 decimal places.
print(f"The accuracy is: {scores.mean():.3f} ± {scores.std():.3f}")

The accuracy is: 0.870 ± 0.002
